In [ ]:
!pip install --upgrade keras tensorflow
!pip install matplotlib
!pip install gdown
!pip3 install -U scikit-learn
!pip install import_ipynb

In [ ]:
!gdown https://drive.google.com/uc?id=1xQqTSUA-celn_KRB9Qpx77g54KR0WHaH
!unzip Leishmania_Treinamento.zip
!rm Leishmania_Treinamento.zip

In [8]:
%load_ext tensorboard

In [1]:
# Imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import math
import random as python_random
import import_ipynb
import AlexNetModel

from os import listdir
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

importing Jupyter notebook from AlexNetModel.ipynb


In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

In [13]:
input_shape = (227, 227, 3)
num_classes = 1
epochs=20
batch_size=5

model_name = "AlexNet-227x227x3-c1-e"+str(epochs)+"-b"+str(batch_size)+"_CLAHE"
train_dir = r'Leishmania_Training_Dataset_CLAHE/treinamento'
test_dir = r'Leishmania_Training_Dataset_CLAHE/validacao'

target_weight_height=(227,227)

alexNetModel = AlexNetModel.AlexNet(input_shape=input_shape,num_classes=num_classes,weights=None)
alexNet = alexNetModel.build()
alexNet.compile(loss=['binary_crossentropy'],
                       optimizer=Adam(learning_rate = 0.00001),
                       metrics=["accuracy"])
alexNet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1_96_11x11_4 (Conv2D)   (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
maxpool_1_3x3_2 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
Conv_2_256_5x5_1 (Conv2D)    (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_3 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
maxpool_2_3x3_2 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
Conv_3_384_3x3_1 (Conv2D)    (None, 13, 13, 384)      

In [14]:
folder_id = time.strftime("%Y_%m_%d-%H_%M_%S")+"_CLAHE"
root_logdir = os.path.join(os.curdir, "logs/info"+folder_id)

def get_run_logdir(folder_id):
    run_id = "run_"+folder_id
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir(folder_id)
tensorboard = TensorBoard(log_dir=run_logdir)

In [15]:
# No Augumentation
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(train_dir, 
                                                target_size=target_weight_height, 
                                                batch_size=batch_size, 
                                                class_mode='binary')

validation_generator = datagen.flow_from_directory(test_dir, 
                                              target_size=target_weight_height, 
                                              batch_size=batch_size, 
                                              class_mode='binary')
print(train_generator.class_indices)


Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
{'Negativo': 0, 'Positivo': 1}


In [16]:
num_train_samples = train_generator.samples
num_val_samples = validation_generator.samples

alexNet.fit(train_generator,
            epochs=epochs,
            validation_data=validation_generator,
            callbacks=[tensorboard])

Epoch 1/20
80/80 [==============================] - 10s 125ms/step - loss: 1.3076 - accuracy: 0.5900 - val_loss: 0.7077 - val_accuracy: 0.5000
Epoch 2/20
80/80 [==============================] - 9s 116ms/step - loss: 0.9672 - accuracy: 0.6550 - val_loss: 0.5866 - val_accuracy: 0.5600
Epoch 3/20
80/80 [==============================] - 9s 116ms/step - loss: 0.8928 - accuracy: 0.7100 - val_loss: 0.6123 - val_accuracy: 0.5700
Epoch 4/20
80/80 [==============================] - 9s 116ms/step - loss: 0.6477 - accuracy: 0.7650 - val_loss: 0.3705 - val_accuracy: 0.8900
Epoch 5/20
80/80 [==============================] - 9s 116ms/step - loss: 0.4482 - accuracy: 0.8250 - val_loss: 0.3166 - val_accuracy: 0.8900
Epoch 6/20
80/80 [==============================] - 9s 115ms/step - loss: 0.3773 - accuracy: 0.8575 - val_loss: 0.1960 - val_accuracy: 0.9600
Epoch 7/20
80/80 [==============================] - 9s 116ms/step - loss: 0.4588 - accuracy: 0.8275 - val_loss: 0.1568 - val_accuracy: 0.9800
Epoch

In [17]:
%tensorboard --logdir $root_logdir

In [ ]:
!gdown https://drive.google.com/uc?id=1tTBldPLMQk8jLLDjBaxwbl_FXpYNS0Ux
!unzip Leishmania_Teste.zip
!rm Leishmania_Teste.zip

In [18]:
# Preprocessamento das imagens
def carregar_imagem(arquivo, image_size):
  img = image.load_img(arquivo, target_size=image_size)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = img/255

  return img

# Carregar imagens em uma lista
def predicao_teste(pasta_teste, image_size):
  imagens = []
  for pasta in listdir(pasta_teste):
    for arquivo in listdir(pasta_teste + '/' + pasta):
      imagens.append(carregar_imagem(pasta_teste + pasta + '/' + arquivo, image_size))
  
  return imagens

In [22]:
imagens = np.vstack(predicao_teste('Leishmania_Test_Dataset/', target_weight_height))
respostas_modelo = alexNet.predict_classes(imagens)
respostas_teste = datagen.flow_from_directory('Leishmania_Test_Dataset/', 
                                              target_size=target_weight_height, 
                                              batch_size=batch_size, 
                                              class_mode='binary')
print('Respostas do modelo\n' + str(np.concatenate(respostas_modelo)))
print('Respostas reais\n' + str(respostas_teste.classes))

# Matriz de confusão
matriz_de_confusao = confusion_matrix(respostas_teste.classes, respostas_modelo)

# Metricas
# Marvin pediu especificamente pra gente avaliar os valores de sensibilidade e
# especificidade
TrueNegative, FalsePositive, FalseNegative, TruePositive = matriz_de_confusao.ravel()

# Precisão
print('Precisao ' + str((TruePositive + TrueNegative) / len(respostas_modelo)))

# Sensibilidade
print('Sensibilidade ' + str(TruePositive / (TruePositive + FalseNegative)))

# Especificidade
print('Especificidade ' + str(TrueNegative / (FalsePositive + TrueNegative)))

Found 44 images belonging to 2 classes.
Respostas do modelo
[0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1
 0 1 0 0 0 0 1]
Respostas reais
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1]
Precisao 0.7272727272727273
Sensibilidade 0.6363636363636364
Especificidade 0.8181818181818182


In [21]:
alexNet.save(f"{model_name}.h5")